In [ ]:
import json
import os
import cv2
import albumentations as A
from pycocotools.coco import COCO

# TODO: Update these paths to your dataset location
image_dir = "./data/raw/"  # 原始图片路径
coco_json_path = "./data/raw/result.json"  # 原始 COCO 标注文件路径
output_image_dir = "./data/processed/augmented_dataset"  # 增强后的图片路径
output_coco_json_path = "./data/processed/augmented_dataset/augmented_annotations.json"  # 增强后的 COCO 标注文件路径

In [2]:

def load_coco_annotations(coco_json_path):
    """
    加载 COCO 格式的标注文件
    """
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)
    return coco_data

def save_coco_annotations(coco_data, output_path):
    """
    保存更新后的 COCO 格式标注文件
    """
    with open(output_path, 'w') as f:
        json.dump(coco_data, f, indent=4)

def augment_coco_dataset(image_dir, coco_json_path, output_image_dir, output_coco_json_path, augmenter, num_augments=50):
    """
    对 COCO 数据集进行增强，并保存增强后的图像和更新的标注
    """
    os.makedirs(output_image_dir, exist_ok=True)

    os.makedirs(output_image_dir + "/images", exist_ok=True)

    # 加载 COCO 标注
    coco = COCO(coco_json_path)
    annotations = coco.dataset['annotations']
    images = coco.dataset['images']

    new_annotations = []
    new_images = []
    annotation_id = max([ann['id'] for ann in annotations]) + 1
    image_id = max([img['id'] for img in images]) + 1

    for img_info in images:
        img_path = os.path.join(image_dir, img_info['file_name'])
        img = cv2.imread(img_path)
        height, width = img.shape[:2]

        # 获取与该图像关联的标注
        img_annotations = [ann for ann in annotations if ann['image_id'] == img_info['id']]

        for i in range(num_augments):  # 生成多个增强版本
            # 应用增强
            bboxes = []
            categories = []
            for ann in img_annotations:
                x, y, w, h = ann['bbox']
                bboxes.append([x, y, x + w, y + h])
                categories.append(ann['category_id'])

            transformed = augmenter(image=img, bboxes=bboxes, category_ids=categories)

            # 保存增强后的图片
            new_file_name = f"{os.path.splitext(img_info['file_name'])[0]}_{i}.png"
            new_image_path = os.path.join(output_image_dir, new_file_name)

            success = cv2.imwrite(new_image_path, transformed['image'])
            if not success:
                print(f"Failed to save image: {new_image_path}")

            # 更新 COCO 图像信息
            new_images.append({
                "id": image_id,
                "file_name": new_file_name,
                "height": height,
                "width": width
            })

            # 更新 COCO 标注信息
            for bbox, category_id in zip(transformed['bboxes'], transformed['category_ids']):
                x_min, y_min, x_max, y_max = bbox
                new_annotations.append({
                    "id": annotation_id,
                    "image_id": image_id,
                    "category_id": category_id,
                    "bbox": [x_min, y_min, x_max - x_min, y_max - y_min],
                    "area": (x_max - x_min) * (y_max - y_min),
                    "iscrowd": 0
                })
                annotation_id += 1

            image_id += 1

    # 更新 COCO 数据
    coco.dataset['images'].extend(new_images)
    coco.dataset['annotations'].extend(new_annotations)

    # 保存增强后的 COCO 标注文件
    save_coco_annotations(coco.dataset, output_coco_json_path)

In [3]:
augmenter = A.Compose([
    # 拉伸和缩放
    A.Affine(scale=(0.8, 1.2), shear=(10, 10), p=0.5),  # 仿射变换，随机缩放和倾斜
    A.Resize(640, 640, p=0.2),  # 将图片调整为固定大小
    A.RandomResizedCrop(height=640, width=640, scale=(0.7, 1.0), ratio=(0.75, 1.333), p=0.5),  # 随机裁剪后缩放

    # 旋转和翻转
    A.Rotate(limit=60, border_mode=0, p=0.7),  # 随机旋转 ±30°，填充超出部分
    A.HorizontalFlip(p=0.5),  # 随机水平翻转

    # 随机仿射变换
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=10, border_mode=0, p=0.5),  # 随机平移、缩放、旋转

    # 色彩和模糊
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),  # 随机亮度和对比度调整
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.3),  # 色调和饱和度调整
    A.GaussianBlur(blur_limit=(3, 7), p=0.3),  # 随机高斯模糊

    # 随机噪声
    A.ISONoise(p=0.3),  # 模拟 ISO 噪声
    A.MultiplicativeNoise(multiplier=(0.9, 1.1), per_channel=True, p=0.3),  # 随机乘性噪声

    # 随机遮挡
    A.CoarseDropout(max_holes=8, max_height=32, max_width=32, fill_value=0, p=0.3),  # 随机遮挡部分区域
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']))

# 执行增强
augment_coco_dataset(image_dir, coco_json_path, output_image_dir, output_coco_json_path, augmenter)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


C:\Users\tfgmo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\albumentations\core\bbox_utils.py:478: RuntimeWarning: invalid value encountered in divide
  & (clipped_box_areas / denormalized_box_areas >= min_visibility - epsilon)
C:\Users\tfgmo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\albumentations\augmentations\dropout\functional.py:210: RuntimeWarning: invalid value encountered in divide
  visibility_ratio = 1 - (intersection_area / box_areas[i])


Failed to save image: C:\Users\tfgmo\OneDrive - Virginia Tech\Mahjong CV\augmented_dataset\images/d4e9ce1c-000015_8.png
